In [1]:
import pandas as pd
import numpy as np
import os
import ast
import argparse
import sys
#from sklearn.model_selection import train_test_split

import processing as pr


Data processing completed.


In [2]:
os.listdir()

['cleaned_data_train.csv',
 '.DS_Store',
 'fetch_data.py',
 'cleaned_data_test.csv',
 'testing.ipynb',
 '__pycache__',
 'phase3_oncology_2011.csv',
 'README.md',
 'phase3_oncology_2011_api.csv',
 '.gitattributes',
 'processing.py']

In [3]:
df = pd.read_csv('phase3_oncology_2011_api.csv')

In [4]:
df

,protocolSection_identificationModule_nctId,protocolSection_identificationModule_orgStudyIdInfo_id,protocolSection_identificationModule_organization_fullName,protocolSection_identificationModule_organization_class,protocolSection_identificationModule_briefTitle,protocolSection_identificationModule_officialTitle,protocolSection_statusModule_statusVerifiedDate,protocolSection_statusModule_overallStatus,protocolSection_statusModule_expandedAccessInfo_hasExpandedAccess,protocolSection_statusModule_startDateStruct_date,...,protocolSection_oversightModule_isUsExport,protocolSection_eligibilityModule_genderBased,protocolSection_eligibilityModule_genderDescription,protocolSection_statusModule_expandedAccessInfo_nctId,protocolSection_statusModule_expandedAccessInfo_statusForNctId,protocolSection_referencesModule_availIpds,protocolSection_identificationModule_nctIdAliases,protocolSection_oversightModule_isUnapprovedDevice,protocolSection_identificationModule_orgStudyIdInfo_type,protocolSection_identificationModule_orgStudyIdInfo_link
0,NCT02794324,CCR3593,Royal Marsden NHS Foundation Trust,OTHER,The HeartSpare Study (Stage I),Optimisation and Individualisation of Heart-Sp...,2016-06,COMPLETED,False,2012-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NCT02125344,GBG 84,German Breast Group,OTHER,"A Phase III Trial Comparing Two Dose-dense, Do...",A Randomized Phase III Trial Comparing Two Dos...,2017-07,COMPLETED,False,2014-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NCT03306992,HCI104671,University of Utah,OTHER,Precision-exercise-prescription for Lung Cance...,A Phase III Randomized Study Comparing the Eff...,2023-06,COMPLETED,False,2017-11-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NCT01901692,AMC IRB 2013-0627,Asan Medical Center,OTHER,Sorafenib vs.TransArterial Chemoembolization P...,Randomized Trial Comparing Sorafenib and Trans...,2017-09,COMPLETED,False,2013-07-29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NCT01154192,081696,"University of California, San Diego",OTHER,Theca Cell Function in Adolescents With Polycy...,Theca Cell Function in Adolescents With Polycy...,2019-04,COMPLETED,False,2011-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629,NCT03825887,CCHE -BM0001,Children's Cancer Hospital Egypt 57357,OTHER,Nalbuphine Versus Morphine for Mucositis Pain ...,Evaluation of Safety and Efficacy of Nalbuphin...,2020-07,COMPLETED,False,2018-02-01,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1630,NCT01962987,71204901,Actavis Inc.,INDUSTRY,A Study Comparing Diclofenac Sodium Gel 3% to ...,"A Randomized, Double-Blind, Multiple-Site, Pla...",2020-07,COMPLETED,False,2013-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1631,NCT01523587,1200.125,Boehringer Ingelheim,INDUSTRY,LUX-Lung 8: A Phase III Trial of Afatinib (BIB...,"LUX-Lung 8: A Randomized, Open-label Phase III...",2019-02,COMPLETED,False,2012-03-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1632,NCT02576054,V501-200,Merck Sharp & Dohme LLC,INDUSTRY,Safety and Tolerability Study of V501 in Japan...,"A Phase III, Open-Label, Clinical Trial to Stu...",2019-11,COMPLETED,False,2015-11-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['start_date'] = df['protocolSection_statusModule_startDateStruct_date'].apply(pr.convert_to_datetime)
df['primary_completion_date'] = df['protocolSection_statusModule_primaryCompletionDateStruct_date'].apply(pr.convert_to_datetime)
df['completion_date'] = df['protocolSection_statusModule_completionDateStruct_date'].apply(pr.convert_to_datetime)
df['primary_study_duration_days'] = (df['primary_completion_date'] - df['start_date']).dt.days
df['study_duration_days'] = (df['completion_date'] - df['start_date']).dt.days

In [6]:
spon_map = {
    'OTHER_GOV': 'OTHER',
    'NETWORK': 'OTHER',
    'NIH': 'OTHER',
    'FED': 'OTHER',
    'INDIV': 'OTHER'
}

spon_map2 = {
    "INDUSTRY": 1,
    "OTHER": 0
}

df['sponsor_type0'] = df['protocolSection_sponsorCollaboratorsModule_leadSponsor_class'].replace(spon_map)
df['sponsor_type'] = df['sponsor_type0'].map(spon_map2)

In [7]:
df['protocolSection_conditionsModule_conditions'] = df['protocolSection_conditionsModule_conditions'].apply(ast.literal_eval)
df['number_of_conditions'] = df['protocolSection_conditionsModule_conditions'].apply(lambda x: len(x))

In [8]:
assign_map = {
    "CROSSOVER": "OTHER",
    "SEQUENTIAL": "OTHER",
    "FACTORIAL": "OTHER"
}

assign_map2 = {
    "SINGLE_GROUP": 0,
    "PARALLEL": 1,
    "OTHER": 2
}

df['intervention_model0'] = df['protocolSection_designModule_designInfo_interventionModel'].replace(assign_map)
df['intervention_model'] = df['intervention_model0'].map(assign_map2)

In [9]:
df['intervention_model'].value_counts(dropna=False)

intervention_model
1.0    1314
0.0     252
2.0      64
NaN       4
Name: count, dtype: int64

In [10]:
# Extract purposes into new columns
df['protocolSection_designModule_designInfo_primaryPurpose'] = df['protocolSection_designModule_designInfo_primaryPurpose'].fillna('')
df['treatment_purpose'] = df['protocolSection_designModule_designInfo_primaryPurpose'].apply(lambda x: 1 if 'TREATMENT' in x else 0)
df['diagnostic_purpose'] = df['protocolSection_designModule_designInfo_primaryPurpose'].apply(lambda x: 1 if 'DIAGNOSTIC' in x else 0)
df['prevention_purpose'] = df['protocolSection_designModule_designInfo_primaryPurpose'].apply(lambda x: 1 if 'PREVENTION' in x else 0)
df['supportive_purpose'] = df['protocolSection_designModule_designInfo_primaryPurpose'].apply(lambda x: 1 if 'SUPPORTIVE_CARE' in x else 0)


In [11]:
# Extract interventions into new columns
df['procedure_intervention'] = df['protocolSection_armsInterventionsModule_interventions'].apply(lambda x: 1 if 'PROCEDURE' in x else 0)
df['device_intervention'] = df['protocolSection_armsInterventionsModule_interventions'].apply(lambda x: 1 if 'DEVICE' in x else 0)
df['behavioral_intervention'] = df['protocolSection_armsInterventionsModule_interventions'].apply(lambda x: 1 if 'BEHAVIORAL' in x else 0)
df['drug_intervention'] = df['protocolSection_armsInterventionsModule_interventions'].apply(lambda x: 1 if 'DRUG' in x else 0)
df['radiation_intervention'] = df['protocolSection_armsInterventionsModule_interventions'].apply(lambda x: 1 if 'RADIATION' in x else 0)
df['biological_intervention'] = df['protocolSection_armsInterventionsModule_interventions'].apply(lambda x: 1 if 'BIOLOGICAL' in x else 0)


In [12]:
df['protocolSection_armsInterventionsModule_armGroups'] = df['protocolSection_armsInterventionsModule_armGroups'].apply(pr.safe_eval)
df['number_of_groups'] = df['protocolSection_armsInterventionsModule_armGroups'].apply(lambda x: len(x) if isinstance(x, list) else 0)


In [13]:
df['protocolSection_armsInterventionsModule_interventions'] = df['protocolSection_armsInterventionsModule_interventions'].apply(eval)
df['intervention_types'] = df['protocolSection_armsInterventionsModule_interventions'].apply(pr.extract_type)
df['number_of_intervention_types'] = df['intervention_types'].apply(len)


In [14]:
age_map = {
    "['ADULT', 'OLDER_ADULT']": "adult",
    "['ADULT']": "adult",
    "['OLDER_ADULT']": "adult",
    "['CHILD']": "youth",
    "['CHILD', 'ADULT']": "youth",
    "['CHILD', 'ADULT', 'OLDER_ADULT']": "all"
}

age_map2 = {
    "youth": 0,
    "adult": 1,
    "all": 2
}
df["age_group0"] = df["protocolSection_eligibilityModule_stdAges"].map(age_map)
df["age_group"] = df["age_group0"].map(age_map2)

In [15]:
df["num_locations"] = df["protocolSection_contactsLocationsModule_locations"].apply(pr.count_loc)


In [16]:
# Combine outcome measures
# May need to separate this and re-do outcome measures features depending on our y variable
df['outcome_measures'] = df['protocolSection_outcomesModule_primaryOutcomes'] + df['protocolSection_outcomesModule_secondaryOutcomes']
df['outcome_measures'] = df['outcome_measures'].astype('str')
df['outcome_measures'] = df['outcome_measures'].str.lower()

df['os_outcome_measure'] = df['outcome_measures'].apply(lambda x: 1 if ' OS ' in x else 0) # sometimes abbreviated as OS
df['os_outcome_measure2'] = df['outcome_measures'].apply(lambda x: 1 if 'overall survival' in x else 0) # sometimes fully spelled out

# Make sure there's no overlap
df['os_outcome_measure'] = df['os_outcome_measure'] + df['os_outcome_measure2']
df = df.drop(columns=['os_outcome_measure2'])
df['os_outcome_measure'] = df['os_outcome_measure'].apply(lambda x: x if x==0 else 1)

In [17]:
df['ae_outcome_measure'] = df['outcome_measures'].apply(lambda x: 1 if 'adverse event' in x else 0)

In [18]:
# Duration of Response OM
df['dor_outcome_measure'] = df['outcome_measures'].apply(lambda x: 1 if ' DOR ' in x else 0) # sometimes abbreviated as DOR
df['dor_outcome_measure2'] = df['outcome_measures'].apply(lambda x: 1 if 'duration of response' in x else 0) # sometimes fully spelled out

# Make sure there's no overlap
df['dor_outcome_measure'] = df['dor_outcome_measure'] + df['dor_outcome_measure2']
df = df.drop(columns=['dor_outcome_measure2'])
df['dor_outcome_measure'] = df['dor_outcome_measure'].apply(lambda x: x if x==0 else 1)

In [19]:
df['protocolSection_outcomesModule_primaryOutcomes'] = df['protocolSection_outcomesModule_primaryOutcomes'].apply(pr.safe_eval)
df['protocolSection_outcomesModule_secondaryOutcomes'] = df['protocolSection_outcomesModule_secondaryOutcomes'].apply(pr.safe_eval)

# Applying the function to create the new column
df['primary_measure'] = df['protocolSection_outcomesModule_primaryOutcomes'].apply(pr.extract_measures)
df['secondary_measure'] = df['protocolSection_outcomesModule_secondaryOutcomes'].apply(pr.extract_measures)

# Applying the function to create the new column
df['primary_timeFrame'] = df['protocolSection_outcomesModule_primaryOutcomes'].apply(pr.extract_timeframes)
df['secondary_timeFrame'] = df['protocolSection_outcomesModule_secondaryOutcomes'].apply(pr.extract_timeframes)

In [20]:
df['primary_duration'] = df['primary_timeFrame'].apply(pr.extract_time_length_from_list)
df['secondary_duration'] = df['secondary_timeFrame'].apply(pr.extract_time_length_from_list)

In [21]:
# Applying the function to the dataframe
df['primary_max_days'] = df['primary_duration'].apply(pr.find_max_duration)
df['secondary_max_days'] = df['secondary_duration'].apply(pr.find_max_duration)

In [22]:
df['protocolSection_oversightModule_oversightHasDmc'] = df['protocolSection_oversightModule_oversightHasDmc'].astype('str')
df['protocolSection_oversightModule_oversightHasDmc'] = df['protocolSection_oversightModule_oversightHasDmc'].str.lower()
dmc_map = {
    'true': 1,
    'false': 0,
}
df['has_dmc'] = df['protocolSection_oversightModule_oversightHasDmc'].map(dmc_map)

In [23]:
party_map = {
    "PRINCIPAL_INVESTIGATOR": 0,
    "SPONSOR": 1,
    "SPONSOR_INVESTIGATOR": 2
}

df['resp_party'] = df['protocolSection_sponsorCollaboratorsModule_responsibleParty_type'].map(party_map)

In [24]:
allo_map = {
    'NON_RANDOMIZED': 0,
    'RANDOMIZED': 1
}

df['allocation'] = df['protocolSection_designModule_designInfo_allocation'].map(allo_map)

In [25]:
mask_map = {
    "NONE": 0,
    "SINGLE": 1,
    "DOUBLE": 2,
    "TRIPLE": 3,
    "QUADRUPLE": 4
}

df['masking'] = df['protocolSection_designModule_designInfo_maskingInfo_masking'].map(mask_map)

In [26]:
#make a dataframe with just the columns of interest
cols = [
    'primary_study_duration_days',
    'study_duration_days',
    'number_of_conditions',
    'number_of_groups',
    'age_group',
    'num_locations',
    # 'intervention_types',
    'number_of_intervention_types',
    'sponsor_type',
    'intervention_model',
    'resp_party',
    # 'protocolSection_sponsorCollaboratorsModule_responsibleParty_type',
    'has_dmc',
    # 'protocolSection_oversightModule_oversightHasDmc',
    'protocolSection_designModule_phases',
    'allocation',
    # 'protocolSection_designModule_designInfo_allocation',
    # 'protocolSection_designModule_designInfo_primaryPurpose',
    # 'protocolSection_designModule_designInfo_maskingInfo_masking',
    'masking',
    'protocolSection_designModule_enrollmentInfo_count',
    'protocolSection_eligibilityModule_healthyVolunteers',
    'treatment_purpose',
    'diagnostic_purpose',
    'prevention_purpose',
    'supportive_purpose',
    'procedure_intervention',
    'device_intervention',
    'behavioral_intervention',
    'drug_intervention',
    'radiation_intervention',
    'biological_intervention',
    'os_outcome_measure',
    'dor_outcome_measure',
    'ae_outcome_measure',
    'primary_max_days',
    'secondary_max_days'
    # 'protocolSection_outcomesModule_primaryOutcomes',
    # 'protocolSection_outcomesModule_secondaryOutcomes',
    # 'protocolSection_eligibilityModule_eligibilityCriteria'
]

clean_df = df[cols].copy()
clean_df

,primary_study_duration_days,study_duration_days,number_of_conditions,number_of_groups,age_group,num_locations,number_of_intervention_types,sponsor_type,intervention_model,resp_party,...,device_intervention,behavioral_intervention,drug_intervention,radiation_intervention,biological_intervention,os_outcome_measure,dor_outcome_measure,ae_outcome_measure,primary_max_days,secondary_max_days
0,820.0,820.0,1,3,1,1.0,4,0,2.0,1,...,1,0,0,0,0,0,0,0,28.0,730.0
1,701.0,791.0,7,2,1,1.0,8,0,1.0,1,...,0,0,1,0,0,1,0,0,126.0,1825.0
2,1666.0,1666.0,2,2,1,1.0,1,0,1.0,1,...,0,1,0,0,0,0,0,0,180.0,180.0
3,1271.0,1494.0,1,2,1,1.0,2,0,1.0,0,...,0,0,1,1,0,1,0,0,84.0,1460.0
4,1065.0,1127.0,1,3,0,1.0,2,0,1.0,0,...,0,0,1,0,0,0,0,0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629,1064.0,1064.0,2,2,0,1.0,2,0,1.0,1,...,0,0,1,0,0,0,0,1,7.0,7.0
1630,184.0,184.0,1,3,1,31.0,2,1,1.0,1,...,0,0,1,0,0,0,0,0,90.0,NaN
1631,595.0,2123.0,1,2,1,192.0,2,1,1.0,1,...,0,0,1,0,0,1,0,0,1058.0,2089.0
1632,992.0,992.0,2,1,0,1.0,1,1,0.0,1,...,0,0,0,0,1,0,0,1,900.0,900.0


In [27]:
object_columns = list(clean_df.select_dtypes(include=['object']).columns)
object_columns

['protocolSection_designModule_phases']

In [28]:
nan_counts = clean_df.isna().sum()
nan_cols = nan_counts[nan_counts > 0].index.tolist()
nan_cols

['primary_study_duration_days',
 'study_duration_days',
 'num_locations',
 'intervention_model',
 'has_dmc',
 'allocation',
 'masking',
 'protocolSection_designModule_enrollmentInfo_count',
 'primary_max_days',
 'secondary_max_days']

In [29]:
need_cols = [
    'primary_study_duration_days',
    'study_duration_days',
    'num_locations',
    'protocolSection_designModule_enrollmentInfo_count']

clean_df2 = clean_df.dropna(subset=need_cols).copy()
clean_df2


,primary_study_duration_days,study_duration_days,number_of_conditions,number_of_groups,age_group,num_locations,number_of_intervention_types,sponsor_type,intervention_model,resp_party,...,device_intervention,behavioral_intervention,drug_intervention,radiation_intervention,biological_intervention,os_outcome_measure,dor_outcome_measure,ae_outcome_measure,primary_max_days,secondary_max_days
0,820.0,820.0,1,3,1,1.0,4,0,2.0,1,...,1,0,0,0,0,0,0,0,28.0,730.0
1,701.0,791.0,7,2,1,1.0,8,0,1.0,1,...,0,0,1,0,0,1,0,0,126.0,1825.0
2,1666.0,1666.0,2,2,1,1.0,1,0,1.0,1,...,0,1,0,0,0,0,0,0,180.0,180.0
3,1271.0,1494.0,1,2,1,1.0,2,0,1.0,0,...,0,0,1,1,0,1,0,0,84.0,1460.0
4,1065.0,1127.0,1,3,0,1.0,2,0,1.0,0,...,0,0,1,0,0,0,0,0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629,1064.0,1064.0,2,2,0,1.0,2,0,1.0,1,...,0,0,1,0,0,0,0,1,7.0,7.0
1630,184.0,184.0,1,3,1,31.0,2,1,1.0,1,...,0,0,1,0,0,0,0,0,90.0,NaN
1631,595.0,2123.0,1,2,1,192.0,2,1,1.0,1,...,0,0,1,0,0,1,0,0,1058.0,2089.0
1632,992.0,992.0,2,1,0,1.0,1,1,0.0,1,...,0,0,0,0,1,0,0,1,900.0,900.0


In [32]:
nan_counts2 = clean_df2.isna().sum()
nan_cols2 = nan_counts2[nan_counts2 > 0].index.tolist()
nan_cols2

['intervention_model',
 'has_dmc',
 'allocation',
 'primary_max_days',
 'secondary_max_days']

In [33]:
nan_cols2.remove('primary_max_days')
nan_cols2.remove('secondary_max_days')

In [34]:
nan_cols2

['intervention_model', 'has_dmc', 'allocation']

In [35]:
# fill NaNs with mode to preserve data distribution
for column in nan_cols2:
    mode_value = clean_df2[column].mode()[0]  # Calculate the mode
    clean_df2[column] = clean_df2[column].fillna(mode_value)  # Replace NaNs with the 

In [37]:
object_columns

['protocolSection_designModule_phases']

In [40]:
clean_df2[object_columns].isna().sum()

protocolSection_designModule_phases    0
dtype: int64

In [47]:
# one hot encode remaining object columns
encoded_df = pd.get_dummies(clean_df2, columns=object_columns).astype(int)
encoded_df

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [46]:
encoded_df["protocolSection_designModule_phases_['PHASE3']"].astype(int)

0       0
1       1
2       1
3       0
4       1
       ..
1629    1
1630    1
1631    1
1632    1
1633    1
Name: protocolSection_designModule_phases_['PHASE3'], Length: 1549, dtype: int64